In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling1D, Embedding, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity 

In [2]:
sentences=open('input.txt','r')
sentences=[text for text in sentences if text.count(' ')>=2]
sentences

['The speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19. \\n\\nFurther, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission Ã¢â‚¬â€œtransmission of the virus before the appearance of symptoms Ã¢â‚¬â€œ is a major driver of transmission for influenza. In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission. \\n\\nThe reproductive number Ã¢â‚¬â€œ the number of secondary infections generated from one infected individual Ã

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1
window_size = 3
tokenized_sentences = tokenizer.texts_to_sequences(sentences)
print(tokenized_sentences)
data, labels = [], []
for sentence in tokenized_sentences:
    for i, target_word in enumerate(sentence):
        context = [
            sentence[j] for j in range(i - window_size, i + window_size + 1)
            if j != i and 0 <= j < len(sentence)
        ]
        data.append(context)
        labels.append(target_word)

data = pad_sequences(data)
print(data)
labels = np.array(labels)
print(data)

[[1, 39, 2, 8, 9, 40, 41, 42, 2, 43, 13, 1, 44, 23, 3, 45, 11, 24, 46, 47, 48, 1, 14, 25, 49, 10, 26, 2, 27, 12, 11, 24, 15, 16, 1, 14, 13, 50, 51, 17, 4, 5, 6, 1, 15, 16, 7, 4, 5, 6, 9, 52, 10, 18, 19, 53, 20, 28, 7, 3, 6, 1, 15, 16, 9, 29, 20, 30, 54, 31, 3, 32, 55, 56, 17, 4, 5, 33, 57, 8, 34, 1, 58, 29, 19, 20, 2, 59, 60, 61, 62, 63, 8, 64, 2, 1, 6, 65, 1, 26, 2, 27, 21, 9, 11, 35, 36, 2, 8, 7, 3, 34, 66, 28, 67, 22, 68, 31, 69, 22, 70, 71, 32, 72, 4, 5, 6, 73, 74, 75, 76, 10, 77, 78, 79, 80, 30, 81, 82, 83, 10, 18, 11, 35, 36, 2, 8, 33, 84, 85, 37, 21, 1, 37, 2, 86, 87, 88, 25, 89, 90, 91, 21, 9, 92, 10, 18, 13, 38, 12, 38, 19, 7, 4, 5, 6, 93, 17, 7, 3, 94, 95, 7, 96, 4, 5, 12, 3, 23, 22, 97, 98, 12, 14, 99, 100, 101, 102, 103, 104]]
[[  0   0   0  39   2   8]
 [  0   0   1   2   8   9]
 [  0   1  39   8   9  40]
 ...
 [  0  99 100 101 103 104]
 [  0   0 100 101 102 104]
 [  0   0   0 101 102 103]]
[[  0   0   0  39   2   8]
 [  0   0   1   2   8   9]
 [  0   1  39   8   9  40]
 .

In [8]:

model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=total_words,output_dim=50,input_length=window_size*2))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(total_words,activation="softmax"))

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.fit(data,labels,epochs=200)

Epoch 1/200


7/7 [==============================] - 1s 5ms/step - loss: 4.6485 - accuracy: 0.0300
Epoch 2/200
7/7 [==============================] - 0s 3ms/step - loss: 4.5434 - accuracy: 0.1700
Epoch 3/200
7/7 [==============================] - 0s 4ms/step - loss: 4.4040 - accuracy: 0.1850
Epoch 4/200
7/7 [==============================] - 0s 2ms/step - loss: 4.2109 - accuracy: 0.1850
Epoch 5/200
7/7 [==============================] - 0s 2ms/step - loss: 3.9602 - accuracy: 0.1850
Epoch 6/200
7/7 [==============================] - 0s 3ms/step - loss: 3.6801 - accuracy: 0.2000
Epoch 7/200
7/7 [==============================] - 0s 3ms/step - loss: 3.4112 - accuracy: 0.2300
Epoch 8/200
7/7 [==============================] - 0s 2ms/step - loss: 3.1484 - accuracy: 0.2550
Epoch 9/200
7/7 [==============================] - 0s 3ms/step - loss: 2.8934 - accuracy: 0.2900
Epoch 10/200
7/7 [==============================] - 0s 4ms/step - loss: 2.6376 - accuracy: 0.3450
Epoch 11/200
7/7 [=========

In [13]:
embeddings=model.layers[0].get_weights()[0]
from sklearn.metrics.pairwise import cosine_similarity
target_word='influenza'
target_embed=embeddings[tokenizer.word_index[target_word]]
similarity=cosine_similarity(target_embed.reshape(1,-1),embeddings)[0]
most_similar=similarity.argsort()[-5:][::-1]
most_similar_words = [word for word, idx in tokenizer.word_index.items() if idx in most_similar]

print(f"Most similar words to '{target_word}': {most_similar_words}")

[array([[-0.848427  ,  0.10762294, -0.8298384 , ...,  0.55546075,
        -0.359472  , -1.0768682 ],
       [ 0.8133237 ,  1.0264677 ,  1.0972701 , ...,  0.04246515,
         0.2447602 , -0.8360855 ],
       [-1.2968448 , -0.10013754, -0.31023276, ..., -0.6578392 ,
         0.9061753 , -0.88982695],
       ...,
       [-1.5368404 ,  0.9094718 ,  0.97930056, ..., -0.0898222 ,
        -0.79439914, -0.56336427],
       [-0.7535901 ,  0.03169273,  1.6239545 , ...,  0.55827844,
        -1.3473465 , -1.9792801 ],
       [ 0.07738912,  0.5531785 ,  0.5367297 , ..., -1.3684158 ,
        -0.43637234, -1.461268  ]], dtype=float32)]
Most similar words to 'influenza': ['influenza', 'we', 'learning', 'present', 'context']
